In [ ]:
def process_image(image_2d, crop_size, last_coords, threshold_mean_intensity=30):
    blurred_image = cv2.GaussianBlur(image_2d, (9, 9), 0)
    crop_half = crop_size // 2

    if last_coords is not None:
        # Define the size of the search box around the last coordinates
        search_box_size = 30
        row_center, col_center = last_coords
        row_start = max(0, row_center - search_box_size)
        row_end = min(image_2d.shape[0], row_center + search_box_size)
        col_start = max(0, col_center - search_box_size)
        col_end = min(image_2d.shape[1], col_center + search_box_size)

        # Extract the sub-image around the last coordinates
        temp_frame = blurred_image[row_start:row_end, col_start:col_end]

        # Find the highest intensity pixel in the sub-image
        max_intensity = np.max(temp_frame)
        max_intensity_index = np.argmax(temp_frame)
        row_sub_center, col_sub_center = np.unravel_index(max_intensity_index, temp_frame.shape)

        # Convert sub-image coordinates to original image coordinates
        row_center = row_start + row_sub_center
        col_center = col_start + col_sub_center

        # Determine the cropping boundaries
        row_start = max(0, row_center - crop_half)
        row_end = min(image_2d.shape[0], row_center + crop_half)
        col_start = max(0, col_center - crop_half)
        col_end = min(image_2d.shape[1], col_center + crop_half)

        # Ensure the crop size is 100x100
        if row_end - row_start < crop_size:
            if row_start == 0:
                row_end = min(crop_size, image_2d.shape[0])
            else:
                row_start = max(0, row_end - crop_size)
        if col_end - col_start < crop_size:
            if col_start == 0:
                col_end = min(crop_size, image_2d.shape[1])
            else:
                col_start = max(0, col_end - crop_size)

        # Crop the image
        cropped_image = image_2d[row_start:row_end, col_start:col_end]

        # Check the mean intensity of the cropped image
        mean_intensity = np.mean(cropped_image)
    if mean_intensity < threshold_mean_intensity or last_coords==None:
        # If last_coords is None, perform a global search
        max_intensity = np.max(blurred_image)
        max_intensity_index = np.argmax(blurred_image)
        row_center, col_center = np.unravel_index(max_intensity_index, image_2d.shape)

        # Recalculate the cropping boundaries
        row_start = max(0, row_center - crop_half)
        row_end = min(image_2d.shape[0], row_center + crop_half)
        col_start = max(0, col_center - crop_half)
        col_end = min(image_2d.shape[1], col_center + crop_half)

        # Ensure the crop size is 100x100
        if row_end - row_start < crop_size:
            if row_start == 0:
                row_end = min(crop_size, image_2d.shape[0])
            else:
                row_start = max(0, row_end - crop_size)
        if col_end - col_start < crop_size:
            if col_start == 0:
                col_end = min(crop_size, image_2d.shape[1])
            else:
                col_start = max(0, col_end - crop_size)

        # Crop the image
        cropped_image = image_2d[row_start:row_end, col_start:col_end]
        #plt.imshow(cropped_image)
        #plt.show()
        #plt.imshow(image_2d)
        #plt.show()

    # Return the coordinates of the max intensity pixel
    return [row_center, col_center], cropped_image

# Open the HDF5 file in read mode
file_path = '/content/drive/MyDrive/MeltPoolViT/Final_DatasetV2.h5'

with h5py.File(file_path, 'r') as file:
    # Access the dataset
    dataset = file['source3']['B8']['rawdata']  # Adjust the path as needed

    # Initialize the previous coordinates
    last_coords = None

    # Initialize list to store cropped images
    cropped_images = []

    # Loop through the specified frames
    for i in range(0, 10240):
        # Select an index to print
        image_data = dataset[i, :, :]

        # If it's the first frame, find the initial max intensity pixel in the whole image
        if last_coords is None:
            max_intensity_index = np.argmax(image_data)
            last_coords = np.unravel_index(max_intensity_index, image_data.shape)

        # Process the image and get the coordinates of the highest intensity pixel
        current_coords, ima = process_image(image_data, 100, last_coords)

        # Append the cropped image to the list
        cropped_images.append(ima)

        # Update the previous coordinates
        last_coords = current_coords

    # Convert the list of cropped images to a NumPy array
    cropped_images_array = np.array(cropped_images)